# Data Cleaning

### 1. Import libraries necessary for the project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from scipy import stats
import statsmodels.formula.api as smf

In [2]:
#plot style
plt.style.use("seaborn")

***

### 2. Import the datasets for analysis

In [3]:
df_risk = pd.read_csv("RISKFACTORSANDACCESSTOCARE.csv")
df_dmg = pd.read_csv("DEMOGRAPHICS.csv")
df_health_summary = pd.read_csv("SUMMARYMEASURESOFHEALTH.csv")

***

### 3. Merge all all three imported data tables 

In [4]:
raw_df = df_risk.merge(df_dmg,how="inner")
raw_df = raw_df.merge(df_health_summary,how="inner")
print(f"size of index: {raw_df.shape[1]}\n")
print(raw_df.columns)

size of index: 91

Index(['State_FIPS_Code', 'County_FIPS_Code', 'CHSI_County_Name',
       'CHSI_State_Name', 'CHSI_State_Abbr', 'Strata_ID_Number', 'No_Exercise',
       'CI_Min_No_Exercise', 'CI_Max_No_Exercise', 'Few_Fruit_Veg',
       'CI_Min_Fruit_Veg', 'CI_Max_Fruit_Veg', 'Obesity', 'CI_Min_Obesity',
       'CI_Max_Obesity', 'High_Blood_Pres', 'CI_Min_High_Blood_Pres',
       'CI_Max_High_Blood_Pres', 'Smoker', 'CI_Min_Smoker', 'CI_Max_Smoker',
       'Diabetes', 'CI_Min_Diabetes', 'CI_Max_Diabetes', 'Uninsured',
       'Elderly_Medicare', 'Disabled_Medicare', 'Prim_Care_Phys_Rate',
       'Dentist_Rate', 'Community_Health_Center_Ind', 'HPSA_Ind',
       'Strata_Determining_Factors', 'Number_Counties', 'Population_Size',
       'Min_Population_Size', 'Max_Population_Size', 'Population_Density',
       'Min_Population_Density', 'Max_Population_Density', 'Poverty',
       'Min_Poverty', 'Max_Poverty', 'Age_19_Under', 'Min_Age_19_Under',
       'Max_Age_19_Under', 'Age_19_64', 'Min

#### There are quite a lot of columns. We will for now focus on the ones we need.

### 4. Lowercase all the column names and filter out unused columns

In [5]:
new_colnames = [raw_df.columns[i].lower() for i in range(len(raw_df.columns))]
raw_df.columns = new_colnames
# Filter out the columns for the confidence interval (CI), max, and min
full_df = raw_df.drop(raw_df.filter(regex ="\Aci_|\Amax|\Amin").columns, axis=1)
print(f"size of filtered index: {full_df.shape[1]}\n")
print(full_df.columns)

size of filtered index: 41

Index(['state_fips_code', 'county_fips_code', 'chsi_county_name',
       'chsi_state_name', 'chsi_state_abbr', 'strata_id_number', 'no_exercise',
       'few_fruit_veg', 'obesity', 'high_blood_pres', 'smoker', 'diabetes',
       'uninsured', 'elderly_medicare', 'disabled_medicare',
       'prim_care_phys_rate', 'dentist_rate', 'community_health_center_ind',
       'hpsa_ind', 'strata_determining_factors', 'number_counties',
       'population_size', 'population_density', 'poverty', 'age_19_under',
       'age_19_64', 'age_65_84', 'age_85_and_over', 'white', 'black',
       'native_american', 'asian', 'hispanic', 'ale', 'us_ale', 'all_death',
       'us_all_death', 'health_status', 'us_health_status', 'unhealthy_days',
       'us_unhealthy_days'],
      dtype='object')


### 5. Check the numbers of population and counties in the original data

In [6]:
print(f"Original population size: {full_df.population_size.sum()}")
print(f"Original number of counties: {full_df.shape[0]}")
print("Original population mean of county: {:.1f}".format(full_df.population_size.mean()))
print("Original population median of county: {:.1f}".format(full_df.population_size.median()))

Original population size: 296410404
Original number of counties: 3141
Original population mean of county: 94368.2
Original population median of county: 25235.0


### 6. Remove entries with missing value

In [7]:
# store variables we are interested in into another dataframe
df = full_df[["no_exercise","diabetes","obesity","few_fruit_veg","smoker","high_blood_pres","poverty","population_size","chsi_state_name","all_death"]]

df = df[(df["no_exercise"]>0) & (df["diabetes"]>0) & (df["obesity"]>0) 
        & (df["few_fruit_veg"]>0) & (df["smoker"]>0) & (df["high_blood_pres"]>0) & (df["poverty"]>0)]

df.head()

,no_exercise,diabetes,obesity,few_fruit_veg,smoker,high_blood_pres,poverty,population_size,chsi_state_name,all_death
0,27.8,14.2,24.5,78.6,26.6,29.1,10.4,48612,Alabama,1041.5
1,27.2,7.2,23.6,76.2,24.6,30.5,10.2,162586,Alabama,856.9
7,29.2,11.2,27.0,81.9,25.5,33.2,16.4,112141,Alabama,1100.3
10,30.3,6.0,31.2,82.8,30.0,26.5,14.9,41744,Alabama,1040.4
21,29.4,10.0,22.2,76.2,27.6,35.0,13.3,79886,Alabama,1028.7


Units for the columns no_exercise, diabetes,obesity, few_fruit_veg, smoker, high_blood_pres, and poverty are in percentage(%).

Unit for population_size is in number of people

chsi_state_name specifies the state in which each of the county is located.

### 7. Print the counts of counties, population size and population means of the counties

In [8]:
print(f"Population size: {df.population_size.sum()}")
print(f"Number of counties: {df.shape[0]}")
print(f"Population mean of county: {round(df.population_size.mean(),0)}")
print(f"Population median of county: {round(df.population_size.median(),0)}")
print(f"Number of States included: {df.chsi_state_name.nunique()}")

Population size: 261559783
Number of counties: 1434
Population mean of county: 182399.0
Population median of county: 65142.0
Number of States included: 50


#### Insight
It is fortunate that after eliminating rows with no entries, the total population didn't drop as much as the proportional decrease in the number of counties. This might be due to that fact that those counties have an average of less people and they do not have suffcient resource and staffs to keep track of these numbers as well as organizing them. This is also evident in the fact that population mean per county almost doubled and median almost tripled after cleaning our data. We conclude that counties with insufficient data have an average of fewer people compare the the entire population. By cleaning them out, we have potentially discarded data that contain bias related to small population.

### 8. Export the cleaned dataset

In [9]:
df.to_csv (r'final-dataset.csv', index = False, header=True)